In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=UOPgHiKSrHw4u7dhvmdmkeY75hjtHo&prompt=consent&access_type=offline&code_challenge=uljd91snBEaLAEFHKinbY2AzpzUpI46GXUZTzra05fA&code_challenge_method=S256

Enter authorization code: 4/0ATx3LY5MLw9Le9EovC6hRi-x3xxH2gLXL6_Kn404ED4SW47pFLztK8Q7sMQGkz0-L0s4XA

You are now logged in as [jonathan@integra-tx.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


# For this fine-tuning you must be connected to at least an A100 GPU with 40GB

In [ ]:
!nvidia-smi

Wed Jun 12 08:01:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              41W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Load necessary files

In [ ]:
# Ortholog sequences to fine-tune the model with
!gsutil cp gs://pt_engineering_bucket/data/Sequences/PiggyBac/clusterRes_no-n_rep_seq_v2.fasta .

Copying gs://pt_engineering_bucket/data/Sequences/PiggyBac/clusterRes_no-n_rep_seq_v2.fasta...
/ [1 files][  9.1 MiB/  9.1 MiB]                                                
Operation completed over 1 objects/9.1 MiB.                                      


# Load Model and set up tokenizer

### Import dependencies:

In [ ]:
!pip install pyarrow==14.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 1.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 14.0.0 which is incompatible.


In [ ]:
!pip install transformers #==4.16.2
!pip install tokenizers #==0.10.3
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install biopython
!git clone https://github.com/salesforce/progen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.1 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Cloning into 'progen'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 248 (delta 9), reused 10 (delta 3), pack-reused 227
Receiving objects: 100% (248/248), 58.62 KiB | 1.78 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [ ]:
from transformers import PreTrainedTokenizerFast
import math
import torch
from tokenizers import Tokenizer
from datasets import load_dataset
from progen.progen2.models.progen.modeling_progen import ProGenForCausalLM
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Download correct model:

In [ ]:
!wget https://storage.googleapis.com/sfr-progen-research/checkpoints/progen2-base.tar.gz
!mkdir -p progen2-base
!tar -xvf progen2-base.tar.gz -C progen2-base

--2024-05-14 10:22:59--  https://storage.googleapis.com/sfr-progen-research/checkpoints/progen2-base.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.11.207, 74.125.26.207, 172.217.204.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.11.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408351368 (1.3G) [application/x-tar]
Saving to: ‘progen2-base.tar.gz’

progen2-base.tar.gz 100%[===================>]   1.31G  55.9MB/s    in 28s     

2024-05-14 10:23:27 (47.2 MB/s) - ‘progen2-base.tar.gz’ saved [1408351368/1408351368]

./
./config.json
./pytorch_model.bin


In [ ]:
model = ProGenForCausalLM.from_pretrained('progen2-base', torch_dtype=torch.float32).to(device)

# Assuming 'model' is your model variable
is_on_cuda = next(model.parameters()).is_cuda
print(is_on_cuda)

True


### Create Tokenizer

In [ ]:
def create_tokenizer_custom(file):
    with open(file, 'r') as f:
        return Tokenizer.from_str(f.read())

tokenizer = create_tokenizer_custom(file='progen/progen2/tokenizer.json')

tokenizer.save("my-tokenizer.json")
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="my-tokenizer.json")

In [ ]:
fast_tokenizer.eos_token = '<|eos|>'
fast_tokenizer.pad_token = fast_tokenizer.eos_token

# Prepare Sequences

In [ ]:
from Bio import SeqIO
import pandas as pd
from sklearn import datasets
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling
from datasets import concatenate_datasets
from datasets import DatasetDict

In [ ]:
def encode(lines):
    a = tokenizer.encode(str(lines['Sequence']))
    return fast_tokenizer(lines['Sequence'], padding=True, add_special_tokens=True, truncation=True, max_length=1024)

In [ ]:
# Prepare the sequences this is the clean version no duplicates or anything...
fasta_file = "clusterRes_no-n_rep_seq_v2.fasta"

protein_names = []
sequences = []
reverse = True

for record in SeqIO.parse(fasta_file, "fasta"):
    #protein_names.append(record.id)
    seq = str(record.seq)
    modified_seq = '1' + seq + '2'  # Add '1' at the beginning and '2' at the end
    # Limit length of strings to be 1024 which is the LLM limit typically
    if len(modified_seq) > 1024:
      modified_seq = seq[-1024:]
    #sequences.append(modified_seq)
    if reverse:
      protein_names.append(record.id + '_reverse')
      reverse_seq = modified_seq[::-1]
      reverse_seq = '1' + reverse_seq[1:]
      reverse_seq = reverse_seq[:-1] + '2'
      sequences.append(reverse_seq)



# Create a DataFrame with the modified sequences
data = {'Protein_Name': protein_names, 'Sequence': sequences}
df = pd.DataFrame(data)
# Filter out sequences that are too short or too long
# Exclude rows where the length of the string is shorter than 300 or longer than 700
df = df[(df['Sequence'].str.len() >= 300) & (df['Sequence'].str.len() <= 700)]
df.reset_index(drop=True, inplace=True)

In [ ]:
# Convert to Hugginface format and encode sequences so they can be used by the model
dataset = Dataset.from_pandas(df)
dataset.set_transform(encode)
# Split into train and test dataset
tokenized_dataset = dataset.train_test_split(test_size=0.2, seed = 42, shuffle = True)

In [ ]:
# Add wt sequences
wt = 'SQRGPTRMCRNIYDPLLCFKLFFTDEIISEIVKWTNAEISLKRRESMTSATFRDTNEDEIYAFFGILVMTAVRKDNHMSTDDLFDRSLSMVYVSVMSRDRFDFLIRCLRMDDKSIRPTLRENDVFTPVRKIWDLFIHQCIQNYTPGAHLTIDEQLLGFRGRCPFRVYIPNKPSKYGIKILMMCDSGTKYMINGMPYLGRGTQTNGVPLGEYYVKELSKPVRGSCRNITCDNWFTSIPLAKNLLQEPYKLTIVGTVRSNKREIPEVLKNSRSRPVGTSMFCFDGPLTLVSYKPKPAKMVYLLSSCDEDASINESTGKPQMVMYYNQTKGGVDTLDQMCSVMTCSRKTNRWPMALLYGMINIACINSFIIYSHNVSSKGEKVQSRKKFMRNLYMGLTSSFMRKRLEAPTLKRYLRDNISNILPKEVPGTSDDSTEEPVTKKRTYCTYCPSKIRRKASASCKKCKKVICREHNIDMCQSCF'
wt_seq_rev = '1' + wt[::-1] + '2'
wt_df = pd.DataFrame({'Protein_Name': ['WT_rev']*10, 'Sequence': [wt_seq_rev]*10})

wt_dataset = Dataset.from_pandas(wt_df)
# Encode dataset
wt_dataset.set_transform(encode)

In [ ]:
# Concatenate to create new training dataset
combined_train_dataset = concatenate_datasets([
    wt_dataset,
    tokenized_dataset['train']
])

shuffled_train_dataset = combined_train_dataset.shuffle(seed=42)

In [ ]:
# Create the final dataset
final_dataset = DatasetDict({
    'train': shuffled_train_dataset,
    'test': tokenized_dataset['test']
})

In [ ]:
# Create data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=fast_tokenizer, mlm=False)

# Train the model

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="progenft_CN_bias",
    overwrite_output_dir=True,
    num_train_epochs=1.75,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    learning_rate=0.00005,
    load_best_model_at_end = True,
    save_total_limit=2,
    prediction_loss_only=False,
    remove_unused_columns=False,
    logging_steps=100,
    fp16=False,
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 2000,
    save_steps=2000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=final_dataset['train'],
    eval_dataset=final_dataset["test"]
)

is_on_cuda = next(model.parameters()).is_cuda
print(is_on_cuda)

True


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
eval_results = trainer.evaluate()
print(f"Original Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Original Perplexity: 10.69


In [ ]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
2000,1.351300,1.323436
4000,0.950800,1.202045
6000,0.886400,1.128549


TrainOutput(global_step=6197, training_loss=1.2038223244440984, metrics={'train_runtime': 5202.6712, 'train_samples_per_second': 4.764, 'train_steps_per_second': 1.191, 'total_flos': 5.610393534207514e+16, 'train_loss': 1.2038223244440984, 'epoch': 1.75})

In [ ]:
model = ProGenForCausalLM.from_pretrained('progenft_CN_bias/checkpoint-6000/', torch_dtype=torch.float32).to(device)

# Assuming 'model' is your model variable
is_on_cuda = next(model.parameters()).is_cuda
print(is_on_cuda)

True
